In [1]:
import os

In [2]:
%pwd

'/home/towet/Desktop/Inteligent-Radiologist-Assistant/Intelligent-Radiologist-Assistant-Knee-Injury-/research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'/home/towet/Desktop/Inteligent-Radiologist-Assistant/Intelligent-Radiologist-Assistant-Knee-Injury-'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from IntelligentRadiologistAssistant.constants import *
from IntelligentRadiologistAssistant.utils.common import read_yaml, create_directories


In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [10]:
import os
import urllib.request as request 
import zipfile
from IntelligentRadiologistAssistant import logger
from IntelligentRadiologistAssistant.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-15 12:43:36,118: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-15 12:43:36,121: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-15 12:43:36,123: INFO: common: created directory at: artifacts]
[2023-09-15 12:43:36,125: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-15 12:46:17,527: INFO: 1170291011: artifacts/data_ingestion/data.zip download! with following info: 
X-GUploader-UploadID: ADPycduJhesPz4OgCvHJ_MjkBa5cUrZX2bObS8s0hKWDBceYzY7-GWhkU9Ie9cFi9rAS3Wt4bDNZyEWYh3J01Gsro_tJtQ
Expires: Fri, 15 Sep 2023 09:43:39 GMT
Date: Fri, 15 Sep 2023 09:43:39 GMT
Cache-Control: private, max-age=0
Last-Modified: Thu, 03 Aug 2023 14:44:46 GMT
ETag: "c0ad0d14edc9fe132d84857b133ab603"
x-goog-generation: 1691073886231900
x-goog-metageneration: 1
x-goog-stored-content-encoding: identity
x-goog-stored-content-length: 100569765
Content-Type: application/zip
Content-Disposition: attachment; filename="Osteoarthritis detection.v5